**Paweł Pozorski**

__Rozwiązanie pracy domowej nr 5__

## 1. Przygotowanie danych

1. Wykonaj `import` potrzebnych pakietów (oprócz poniżej wymienionych), tak aby umieszczony w tym notebooku kod działał.

In [1]:
import os
import sqlite3
import warnings
import pickle
import time

import pandas as pd
import numpy as np

warnings.simplefilter("ignore")

2. Wczytaj ramki danych, na których będziesz dalej pracował

**UWAGA:**
* Pliki muszą znajdować się w katalogu "travel_stackexchange_com" w tym samym katalogu co ten notebook (plik .ipynb).
* Nazwy tabel muszą być *zgodne z instrukcją zamieszczoną w treści pracy domowej*.


In [2]:
# DOPISZ TU ODPOWIEDNI KOD
ROOT_DIR = "travel_stackexchange_com"


def load_df(df_name, **kwargs):
    return pd.read_csv(
        os.path.join(ROOT_DIR, df_name + ".csv.gz"), compression="gzip", **kwargs
    )


# Comments = ...  # wczytujemy z pliku './travel_stackexchange_com/Comments.csv.gz'
Comments = load_df("Comments")
# Posts = ...  # wczytujemy z pliku './travel_stackexchange_com/Posts.csv.gz'
Posts = load_df("Posts")
# Users = ...  # wczytujemy z pliku './travel_stackexchange_com/Users.csv.gz'
Users = load_df("Users")

3. Przygotuj bazę danych wykonując poniższą komórkę.

In [3]:
# NIE MODYFIKUJ TEJ KOMÓRKI
SCIEZKA_BAZY = "./pd5_baza.db"  # Ścieżka do pliku z bazą danych ('./' oznacza bieżący katalog, czyli będzie to plik w tym samym katalogu, co ten notebook).
with sqlite3.connect(SCIEZKA_BAZY) as conn:  # połączenie do bazy danych
    # wewnątrz bloku `with` mamy dostępny obiekt połączenia, które jest automatycznie zamykane po jego opuszczeniu.
    Comments.to_sql(
        "Comments", conn, if_exists="replace"
    )  # jeżeli ramka danych już istnieje, to jest nadpisywana.
    Posts.to_sql("Posts", conn, if_exists="replace")
    Users.to_sql("Users", conn, if_exists="replace")

## 2. Wyniki zapytań SQL

Wykonaj zapytania sql. Poniższy kod zapisze też wynik do pliku bazy - potem można go z niej odczytać.

**Uwaga**: Zapytania powinny się wykonywać nie dłużej niż kilka sekund każde,
jednak czasem występują problemy zależne od systemu, np. pod Linuxem zapytania 3 i 5 potrafią zająć
odp. kilka minut i ponad godzinę. Żeby obejść ten problem pod koniec tej sekcji pokazane jest, jak
zapisać wyniki zapytań do tymczasowych plików `pickle`.


In [4]:
# NIE MODYFIKUJ TEJ KOMÓRKI (poza ew. zakomentowaniem wywoływania zapytań)
zapytanie_1 = """
SELECT Location, SUM(UpVotes) as TotalUpVotes
FROM Users
WHERE Location != ''
GROUP BY Location
ORDER BY TotalUpVotes DESC
LIMIT 10
"""

zapytanie_2 = """
SELECT STRFTIME('%Y', CreationDate) AS Year,
       STRFTIME('%m', CreationDate) AS Month,
       COUNT(*) AS PostsNumber, MAX(Score) AS MaxScore
FROM Posts
WHERE PostTypeId IN (1, 2)
GROUP BY Year, Month
HAVING PostsNumber > 1000
"""

Questions_query = """
SELECT OwnerUserId, SUM(ViewCount) as TotalViews
FROM Posts
WHERE PostTypeId = 1
GROUP BY OwnerUserId
"""
zapytanie_3 = """
SELECT Id, DisplayName, TotalViews
FROM Questions
JOIN Users
ON Users.Id = Questions.OwnerUserId
ORDER BY TotalViews DESC
LIMIT 10
"""

zapytanie_4 = """
SELECT DisplayName, QuestionsNumber, AnswersNumber, Location, Reputation, UpVotes, DownVotes
FROM (
        SELECT *
        FROM (
                SELECT COUNT(*) as AnswersNumber, OwnerUserId
                FROM Posts
                WHERE PostTypeId = 2
                GROUP BY OwnerUserId
            ) AS Answers
        JOIN
            (
                SELECT COUNT(*) as QuestionsNumber, OwnerUserId
                FROM Posts
                WHERE PostTypeId = 1
                GROUP BY OwnerUserId
            ) AS Questions
        ON Answers.OwnerUserId = Questions.OwnerUserId
        WHERE AnswersNumber > QuestionsNumber
        ORDER BY AnswersNumber DESC
        LIMIT 5
    ) AS PostsCounts
JOIN Users
ON PostsCounts.OwnerUserId = Users.Id
"""

CmtTotScr_query = """
SELECT PostId, SUM(Score) AS CommentsTotalScore 
FROM Comments
GROUP BY PostId
"""
PostsBestComments_query = """
SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount, CmtTotScr.CommentsTotalScore
FROM CmtTotScr
JOIN Posts ON Posts.Id = CmtTotScr.PostId 
WHERE Posts.PostTypeId=1
"""
zapytanie_5 = """
SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location 
FROM PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id 
ORDER BY CommentsTotalScore DESC
LIMIT 10
"""


def wykonaj_zapytanie(zap, k, conn_):
    """
    Wykonuje zapytanie zap (str) o numerze k (int) / nazwie k (str) przy użyciu połączenia z bazą w obiekcie conn_.
    Wypisuje czas wykonania zapytania w sekundach (dokładnie tzw. Wall Clock Time, czyli czas rzeczywisty, który upłynął).
    Zwraca wynik zapytania jako ramkę danych Pandas.
    """
    t0 = time.time()  # zapisanie obecnego czasu do zmiennej
    wynik = pd.read_sql_query(zap, conn_)  # wykonanie zapytania
    t = time.time() - t0  # czas (w sekundach), jaki zajęło wykonania zapytania
    print(f"Wykonano zapytanie {k} w {t:.2f}s.")
    return wynik


# Poniższy blok with wykonuje pomocnicze zapytania;
with sqlite3.connect(SCIEZKA_BAZY) as conn:
    wykonaj_zapytanie(Questions_query, "Questions", conn).to_sql(
        "Questions", conn, if_exists="replace"
    )
    wykonaj_zapytanie(CmtTotScr_query, "CmtTotScr", conn).to_sql(
        "CmtTotScr", conn, if_exists="replace"
    )
    wykonaj_zapytanie(PostsBestComments_query, "PostsBestComments", conn).to_sql(
        "PostsBestComments", conn, if_exists="replace"
    )

# Poniższy blok with wykonuje wszystkie 5 zapytań;
# Wyniki umieszcza w zmiennych sql_i.
with sqlite3.connect(SCIEZKA_BAZY) as conn:
    sql_1 = wykonaj_zapytanie(zapytanie_1, 1, conn)
    sql_2 = wykonaj_zapytanie(zapytanie_2, 2, conn)
    sql_3 = wykonaj_zapytanie(zapytanie_3, 3, conn)
    sql_4 = wykonaj_zapytanie(zapytanie_4, 4, conn)
    sql_5 = wykonaj_zapytanie(zapytanie_5, 5, conn)

Wykonano zapytanie Questions w 0.08s.
Wykonano zapytanie CmtTotScr w 0.16s.
Wykonano zapytanie PostsBestComments w 0.13s.
Wykonano zapytanie 1 w 0.03s.
Wykonano zapytanie 2 w 0.18s.
Wykonano zapytanie 3 w 0.05s.
Wykonano zapytanie 4 w 0.22s.
Wykonano zapytanie 5 w 0.07s.


Moduł `pickle` to system serializacji (konwersji danych do postaci mogącej być zapisaną na dysku/przesłaną przez sieć) obiektów Pythona.
W przeciwieństwie do plików np `csv` można zapisywać niemal dowonly obiekt Pythona. Minusem jest niemożliwość przeczytania takiego obiektu inaczej, niż przez program napisany w Pythonie.

Poniższy kod zapisze wyniki zapytań (ramiki danych Pandas) do takich plików, aby można je potem było szybko odczytać.
**Nie trzeba z niego korzystać, jeżeli nie ma problemów z wykonaniem zapytań SQL.**

In [5]:
# Zapisanie każdej z ramek danych opisujących wyniki zapytań SQL do osobnego pliku pickle.
for i, df in enumerate([sql_1, sql_2, sql_3, sql_4, sql_5], 1):
    df.to_pickle(f"sql_{i}.pkl.gz")

In [6]:
# Wczytanie policzonych uprzednio wyników z plików pickle (możesz to zrobić, jeżeli zapytania wykonują się za długo).
sql_1, sql_2, sql_3, sql_4, sql_5 = [
    pd.read_pickle(f"sql_{i}.pkl.gz") for i in range(1, 5 + 1)
]

## 3. Wyniki zapytań SQL odtworzone przy użyciu metod pakietu Pandas.

Wynikowa ramka danych do zapytania 1 popwinna nazwyać się `pandas_1`, do drugiego `pandas_2` itd.

### Zadanie 1

In [7]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_1

    pandas_1 = (
        Users[Users["Location"] != ""]
        .groupby("Location")["UpVotes"]
        .sum()
        .rename("TotalUpVotes")
        .sort_values(ascending=False)
        .reset_index()
        .head(10)
    )

    # sprawdzenie równoważności wyników
    print(pandas_1.equals(sql_1))

except Exception as e:
    print("Zad. 1: niepoprawny wynik.")
    print(e)

True


### Zadanie 2

In [8]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_2

    pandas_2 = Posts[Posts["PostTypeId"].isin([1, 2])][["CreationDate", "Score"]]

    CreationDate = pd.to_datetime(pandas_2["CreationDate"])
    pandas_2["Year"] = CreationDate.dt.strftime("%Y")
    pandas_2["Month"] = CreationDate.dt.strftime("%m")

    pandas_2 = pandas_2.groupby(["Year", "Month"])
    MaxScore = pandas_2["Score"].max().reset_index(drop=True)
    pandas_2 = (
        pandas_2["Score"].count().reset_index().rename(columns={"Score": "PostsNumber"})
    )
    pandas_2["MaxScore"] = MaxScore

    pandas_2 = pandas_2[pandas_2["PostsNumber"] > 1000].reset_index(drop=True)

    # sprawdzenie równoważności wyników
    print(pandas_2.equals(sql_2))

except Exception as e:
    print("Zad. 2: niepoprawny wynik.")
    print(e)

True


### Zadanie 3

In [9]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_3

    Questions = (
        Posts[Posts["PostTypeId"] == 1][["OwnerUserId", "ViewCount"]]
        .groupby("OwnerUserId")["ViewCount"]
        .sum()
        .rename("TotalViews")
        .reset_index()
    )
    pandas_3 = (
        pd.merge(Questions, Users, left_on="OwnerUserId", right_on="Id")[
            ["Id", "DisplayName", "TotalViews"]
        ]
        .sort_values("TotalViews", ascending=False)
        .head(10)
        .reset_index(drop=True)
    )

    # sprawdzenie równoważności wyników
    print(pandas_3.equals(sql_3))

except Exception as e:
    print("Zad. 3: niepoprawny wynik.")
    print(e)

True


### Zadanie 4

In [10]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_4

    Answers = (
        Posts[Posts["PostTypeId"] == 2]
        .groupby("OwnerUserId")["OwnerUserId"]
        .count()
        .rename("AnswersNumber")
        .reset_index()
    )
    Questions = (
        Posts[Posts["PostTypeId"] == 1]
        .groupby("OwnerUserId")["OwnerUserId"]
        .count()
        .rename("QuestionsNumber")
        .reset_index()
    )
    PostsCounts = pd.merge(Answers, Questions, on="OwnerUserId")
    PostsCounts = (
        PostsCounts[PostsCounts["AnswersNumber"] > PostsCounts["QuestionsNumber"]]
        .sort_values("AnswersNumber", ascending=False)
        .head(5)
        .reset_index(drop=True)
    )
    pandas_4 = pd.merge(PostsCounts, Users, left_on="OwnerUserId", right_on="Id")[
        [
            "DisplayName",
            "QuestionsNumber",
            "AnswersNumber",
            "Location",
            "Reputation",
            "UpVotes",
            "DownVotes",
        ]
    ]

    # sprawdzenie równoważności wyników
    print(pandas_4.equals(sql_4))

except Exception as e:
    print("Zad. 4: niepoprawny wynik.")
    print(e)

True


### Zadanie 5

In [11]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_5

    pandas_5 = CmtTotScr = (
        Comments[["PostId", "Score"]]
        .groupby("PostId")["Score"]
        .sum()
        .rename("CommentsTotalScore")
        .reset_index()
    )
    PostsBestComments = Posts[Posts["PostTypeId"] == 1].merge(
        CmtTotScr, left_on="Id", right_on="PostId"
    )[["OwnerUserId", "Title", "CommentCount", "ViewCount", "CommentsTotalScore"]]
    pandas_5 = (
        pd.merge(Users, PostsBestComments, left_on="Id", right_on="OwnerUserId")[
            [
                "Title",
                "CommentCount",
                "ViewCount",
                "CommentsTotalScore",
                "DisplayName",
                "Reputation",
                "Location",
            ]
        ]
        .sort_values("CommentsTotalScore", ascending=False)
        .head(10)
        .reset_index(drop=True)
    )

    # sprawdzenie równoważności wyników
    print(pandas_5.equals(sql_5))

except Exception as e:
    print("Zad. 5: niepoprawny wynik.")
    print(e)

True
